In [ ]:
from dotenv import load_dotenv

# Load API KEY information
load_dotenv(override=True)

from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate


In [ ]:
#Parameters and ChatMistral object creation


# Create the ChatMistralAI object
llm = ChatMistralAI(
    temperature=1,  # Low temperature for more focused responses
    model="mistral-small-latest", 
)

#If we want to understand pictures, we should use this model : "pixtral-12b-2409"

# CO2 et autres


In [ ]:
# Code to extract token use (exemple)
question = "What is the capital of Japan?"
response = llm.invoke(question)    #Get everything
token_usage = response.response_metadata.get("token_usage", {})


# Time to first token ?


# Prompt engineering

Here we setup a basic template for our prompt engineering.


In [ ]:
template = """You are an expert {role} with 10 years of experience.
Please provide advice on the following topic.

Topic: {topic}

Format your response as:
- Main Advice:
- Key Points:
- Recommendations:
"""

# Create prompt and chain
prompt = PromptTemplate.from_template(template)
model = ChatMistralAI(model="mistral-small-latest", temperature=0.3)
output_parser = StrOutputParser()
parser = StrOutputParser()

chain = prompt | model | output_parser

# Execute the chain
response = chain.invoke({
    "role": "History teacher",
    "topic": "World War 2"
})

print(response)

# Helper to run a template with variables

def run_chain(template: str, variables: dict):
    prompt = PromptTemplate.from_template(template)
    chain = prompt | llm | parser
    return chain.invoke(variables)

There are multiple types of prompt that we could use :
    -Persona
    -Automation
    -CoT
    -Few shot learning
    -Subtasks

In [ ]:
persona_template = (
    "Act as a supportive but rigorous computer science teacher.\n" \
    "Your tone should be constructive, specific, and pedagogical.\n" \
    "You are reviewing a student's code for an assignment.\n" \
    "Assignment requirement: {task_description}\n" \
    "Grading rubric: {grading_rubric}\n" \
    "Deliverables: corrected_code, grade (0-100), advice (array).\n" \
    "\n" \
    "Student code:\n{student_code}\n"
)

# print(run_chain(persona_template, {
#     "task_description": task_description,
#     "grading_rubric": grading_rubric,
#     "student_code": student_code
# }))

json_template = (
    "Act as a supportive but rigorous computer science teacher.\n"
    "You are reviewing a student's code for an assignment.\n"
    "Assignment requirement: {task_description}\n"
    "Grading rubric: {grading_rubric}\n"
    "Return ONLY a JSON object with these keys:\n"
    "- corrected_code: string containing the fixed code\n"
    "- grade: number (0-100), must equal sum of all scores\n"
    "- scores: object with correctness (0-40), robustness (0-25), quality (0-20), efficiency (0-10), style (0-5)\n"
    "- advice: array of short, actionable improvement suggestions\n"
    "Constraints: grade MUST equal correctness+robustness+quality+efficiency+style. No extra text outside the JSON.\n\n"
    "Student code:\n{student_code}\n"
)

# print(run_chain(json_template, {
#     "task_description": task_description,
#     "grading_rubric": grading_rubric,
#     "student_code": student_code
# }))

fewshot_template = (
    "Act as a supportive but rigorous computer science teacher.\n" \
    "You are reviewing a student's code for an assignment.\n" \
    "Assignment requirement: {task_description}\n" \
    "Grading rubric: {grading_rubric}\n" \
    "Deliverables: corrected_code, grade (0-100), advice (array). Return JSON.\n" \
    "\n" \
    "Example 1 — Input code:\n"
    "def add(a,b):\n    return a-b\n" \
    "Example 1 — Desired JSON output:\n"
    "{{\n"
    "  \"corrected_code\": \"def add(a, b):\\n    return a + b\",\n"
    "  \"grade\": 70,\n"
    "  \"advice\": [\"Add docstrings\", \"Use spaces around operators\"]\n"
    "}}\n" \
    "\n" \
    "Example 2 — Input code:\n"
    "def divide(a,b):\n    return a/b\n" \
    "Example 2 — Desired JSON output:\n"
    "{{\n"
    "  \"corrected_code\": \"def divide(a, b):\\n    if b == 0:\\n        raise ZeroDivisionError(\'division by zero\')\\n    return a / b\",\n"
    "  \"grade\": 85,\n"
    "  \"advice\": [\"Handle division by zero\", \"Add type hints\"]\n"
    "}}\n" \
    "\n" \
    "Now review this Student code and return JSON as above:\n{student_code}\n"
)

# print(run_chain(fewshot_template, {
#     "task_description": task_description,
#     "grading_rubric": grading_rubric,
#     "student_code": student_code
# }))


subtasks_template = (
    "Role: supportive, rigorous CS teacher.\n" \
    "Requirement: {task_description}\n" \
    "Rubric: {grading_rubric}\n" \
    "Follow these subtasks:\n" \
    "1) Validate the implementation against the requirement (incl. empty list).\n" \
    "2) List specific defects you find (line- or behavior-based).\n" \
    "3) Provide corrected_code that fully meets the requirement.\n" \
    "4) Assign a grade (0-100) using the rubric.\n" \
    "5) Provide actionable advice as an array of short, concrete suggestions.\n" \
    "Output your reasoning for each task and at the end give a JSON with keys: checks (array), bugs (array), corrected_code (string), grade (number), advice (array).\n" \
    "\n" \
    "Student code:\n{student_code}\n"
)

# print(run_chain(subtasks_template, {
#     "task_description": task_description,
#     "grading_rubric": grading_rubric,
#     "student_code": student_code
# }))


#Chain of Thoughts
print("Without the chain of thought reasoning:")
noCoT =  "A train leaves Station A at 8:00 AM traveling at 60 km/h. Another train leaves Station B at 9:00 AM " \
"traveling at 80 km/h toward Station A. If the stations are 300 km apart, at what time will the two trains meet? " \
"Don't think step-by-step and give no reasoning. Just give the response."
print(run_chain(noCoT, {}))

print("With the chain of thought reasoning:")
CoT =  "A train leaves Station A at 8:00 AM traveling at 60 km/h. Another train leaves Station B at 9:00 AM " \
"traveling at 80 km/h toward Station A. If the stations are 300 km apart, at what time will the two trains meet? "
print(run_chain(CoT, {}))



# Access the web

In [ ]:
import os, json, inspect
from typing import Callable, List
from mistralai import Mistral, UserMessage, ToolMessage


In [ ]:
API_KEY = os.getenv("MISTRAL_API_KEY")
if not API_KEY:
    print("Warning: MISTRAL_API_KEY is not set.")

MODEL_NAME = os.getenv("MISTRAL_MODEL", "mistral-small")
TEMPERATURE = float(os.getenv("MISTRAL_TEMPERATURE", "0.0"))
client = Mistral(api_key=API_KEY)

print(f"Environment loaded! Using model: {MODEL_NAME}")

In [ ]:
def build_tool_spec(func: Callable):
    """Build a tool spec dict from a plain python function.
    Assumes all parameters are strings unless type annotation gives something else.
    """
    sig = inspect.signature(func)
    props = {}
    required = []
    for name, param in sig.parameters.items():
        ann = param.annotation
        ann_type = "string"
        if ann in (int, float):
            ann_type = "number"
        props[name] = {"type": ann_type}
        if param.default is inspect._empty:
            required.append(name)
    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip()[:800],
            "parameters": {
                "type": "object",
                "properties": props,
                "required": required
            }
        }
    }

def run_tool_chat(user_content: str, funcs: List[Callable], model: str = MODEL_NAME, temperature: float = TEMPERATURE):
    """Send a user message, handle any tool calls, return final answer string."""
    messages = [UserMessage(role="user", content=user_content)]
    tool_specs = [build_tool_spec(f) for f in funcs]
    first = client.chat.complete(model=model, messages=messages, tools=tool_specs, temperature=temperature)
    msg = first.choices[0].message
    tool_calls = msg.tool_calls or []
    if not tool_calls:
        return msg.content
    messages.append(msg)
    for tc in tool_calls:
        # Parse args and execute matching function
        args = json.loads(tc.function.arguments)
        fn = next((f for f in funcs if f.__name__ == tc.function.name), None)
        if fn is None:
            result = f"Error: function {tc.function.name} not implemented"
        else:
            try:
                result = fn(**args)
            except Exception as e:
                result = f"Error executing {tc.function.name}: {e}".strip()
        print(f"Tool {tc.function.name}({args}) -> {str(result)[:160]}")
        messages.append(ToolMessage(role="tool", content=str(result), name=tc.function.name, tool_call_id=tc.id))
    final = client.chat.complete(model=model, messages=messages, temperature=temperature)
    return final.choices[0].message.content



When the answer is not in the course, or the amount of info is deemed weak, we can use the foloowing code to get info from wikipedia.

In [ ]:
import wikipedia

def search_wikipedia(query: str) -> str:
    """Search Wikipedia for information about a topic.
    Args:
        query: The search query or topic to look up on Wikipedia.
    Returns:
        Summary of the Wikipedia article or error message.
    """
    try:
        # Set language to English
        wikipedia.set_lang("en")
        # Get summary (first few sentences)
        summary = wikipedia.summary(query, sentences=3)
        return summary
    except wikipedia.exceptions.DisambiguationError as e:
        # Multiple possible pages - return the options
        options = ", ".join(e.options[:5])
        return f"Multiple results found. Please be more specific. Options: {options}"
    except wikipedia.exceptions.PageError:
        return f"No Wikipedia page found for '{query}'"
    except Exception as e:
        return f"Wikipedia search error: {e}"

# Test the tool
print("=== Testing Wikipedia Search Tool ===")
print(search_wikipedia("Artificial Intelligence"))
print()
print(search_wikipedia("Python programming"))

In [ ]:
# Wikipedia search via mistralai tool calling
print("=== Wikipedia Search Tool Call ===")
wiki_answer = run_tool_chat(
    "Does Brussels have a governement?",
    [search_wikipedia]
)
print("Final Answer:", wiki_answer)

# Store results in local database

In [ ]:
# Create a simple SQLite database with student data
import sqlite3

# Create in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create table and insert sample data
cursor.execute('''
CREATE TABLE students (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    grade REAL,
    subject TEXT
)
''')

students_data = [
    (1, 'Alice', 20, 85.5, 'Computer Science'),
    (2, 'Bob', 21, 92.0, 'Mathematics'),
    (3, 'Charlie', 19, 78.5, 'Physics'),
    (4, 'Diana', 22, 88.0, 'Computer Science'),
    (5, 'Eve', 20, 95.5, 'Mathematics'),
]

cursor.executemany('INSERT INTO students VALUES (?,?,?,?,?)', students_data)
conn.commit()

print("Sample database created with 5 students")

In [ ]:
# Database query tool (plain function)
def query_students_db(query: str) -> str:
    """Execute a read-only SELECT query against the Computer Science students in-memory 
    students grade table that was created with 
    CREATE TABLE students (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    grade REAL,
    subject TEXT).
    Args:
        query: SQL SELECT statement.
    Returns:
        Formatted rows or error message.
    """
    try:
        if not query.strip().upper().startswith("SELECT"):
            return "Only SELECT queries allowed"
        cursor.execute(query)
        rows = cursor.fetchall()
        if not rows:
            return "No results"
        return "\n".join(str(r) for r in rows)
    except Exception as e:
        return f"DB error: {e}".strip()

# Quick test
print(query_students_db("SELECT name, grade FROM students WHERE grade > 85"))

In [ ]:
# Database + calculator via mistralai
print("=== Database Tool Call ===")
db_answer = run_tool_chat(
    "What is the average grade of all Computer Science students? Show the numeric average only.",
    [query_students_db, calculator]
)
print("Final Answer:", db_answer)

# Custom API (if the course and wikipedia is not enough)

In [ ]:
import requests, json

# City info tool (plain function)
def get_city_info(city: str) -> str:
    """Lookup basic city info (lat, lon, population) via Open-Meteo geocoding API.
    Args:
        city: City name.
    Returns:
        Multi-line string with details or error.
    """
    try:
        url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1&language=en&format=json"
        r = requests.get(url, timeout=8)
        r.raise_for_status()
        data = r.json()
        results = data.get("results", [])
        if not results:
            return f"City '{city}' not found"
        res = results[0]
        return (
            f"City: {res.get('name')}\n" \
            f"Country: {res.get('country')}\n" \
            f"Latitude: {res.get('latitude')}\n" \
            f"Longitude: {res.get('longitude')}\n" \
            f"Population: {res.get('population', 'N/A')}"
        )
    except Exception as e:
        return f"City info error: {e}".strip()

# Quick test
print(get_city_info("Brussels"))

# Loading course (if PDF)

In [ ]:
# Step 1: Load Documents
loader = PyMuPDFLoader("RéglementECAM.pdf")
docs = loader.load()
print(f"Number of pages in the document: {len(docs)}")

# Step 2: Split Documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"Number of split chunks: {len(split_documents)}")


# Step 3: Generate Embeddings
embeddings = MistralAIEmbeddings(model="mistral-embed")

# Step 4: Create and Save the Database
# Create a vector store
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)
print("Vector store created successfully!")

# Step 5: Create Retriever
# Search and retrieve information contained in the documents
retriever = vectorstore.as_retriever()


# Step 6: Create Prompt


prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# Step 7: Setup LLM
llm = ChatMistralAI(model="mistral-small-latest", temperature=0)

# Step 8: Create Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



#Exemple

# Run Chain
# Input a query about the document and print the response
question = "Where has the application of AI in healthcare been confined to so far?"
response = chain.invoke(question)
print(response)
print("-----")
question = "Qu'est-ce que je risque à plagier un travail académique?"
response = chain.invoke(question)
print(response)
